Installing required modules

In [3]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00


In [9]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [25]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
import torch
import sentencepiece as spm
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
from transformers import T5Config
import PyPDF2
from heapq import nlargest

Creating a function to extract text from document

In [12]:
def text_extract(pdf_file: str) -> [str]:
    with open(pdf_file, 'rb') as pdf:
        reader = PyPDF2.PdfReader(pdf,strict=False)
        pdf_text =[]

        for page in reader.pages:
            content = page.extract_text()
            pdf_text.append(content)
        return pdf_text

Creating a function for Extractive summarization

In [49]:
def Extractive(text):

  stopwords = list(STOP_WORDS)

  nlp = spacy.load('en_core_web_sm')

  doc = nlp(text)

  tokens = [token.text for token in doc]

  word_freq = {}

  for word in doc:

    if word.text.lower() not in stopwords and word.text.lower() not in punctuation:

      if word.text not in word_freq.keys():
        word_freq[word.text] = 1

      else:
        word_freq[word.text] += 1

  max_freq = max(word_freq.values())

  for word in word_freq.keys():
    word_freq[word] = word_freq[word]/max_freq

  sent_tokens = [sent for sent in doc.sents]

  sent_score = {}

  for sent in sent_tokens:

    for word in sent:

      if word.text in word_freq.keys():

        if sent not in sent_score.keys():
          sent_score[sent] = word_freq[word.text]

        else:
          sent_score[sent] += word_freq[word.text]

  select_len = int(len(sent_tokens) * 0.4)

  from heapq import nlargest

  summary = nlargest(select_len,sent_score,key = sent_score.get)

  final_summary = [word.text for word in summary]

  summary = ' '.join(final_summary)

  final_summary = ""

  for i in summary:
     final_summary +=i

  return  final_summary

Creating a function for Abstractive summarization

In [36]:
def Abstractive(text,min,max):

  model = T5ForConditionalGeneration.from_pretrained('t5-large')

  tokenizer = T5Tokenizer.from_pretrained("t5-base", use_fast=True)

  device = torch.device('cpu')

  pre_processed_text= text.strip().replace('\n','')

  t5_input_text = 'summarize:' + pre_processed_text

  tokenized_text = tokenizer.encode(t5_input_text, return_tensors='pt').to(device)

  summary_ids = model.generate(tokenized_text, min_length = min, max_length= max)

  summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)

  return summary

In [39]:
content = text_extract("/content/story.pdf")
msg=""
for i in extracted_info:
    msg +=i
print(Abstractive(msg,300,600))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

a young explorer named Alex set out on a journey to the enchanted forest. armed with a map and a heart full of curiosity, Alex ventured into the woods. deep within the forest, Alex stumbled upon a talking squirrel named Squeaky. Squeaky revealed that the forest was under a spell that could only be broken by finding a rare, shimmering flower. the flower's petals had the power to li the enchantment and a a a a a a a a a  a a a a a   a    . a. a      .. .. .. .. the...... a.  the


In [54]:
print("please select you input format \n press - one for -string \n press - two - for - documnet")

choice = input("please enter your choice : ")

match choice:
  case "one" : text = input("Enter your text here : ")

  case "two" : text = text_extract("/content/story.pdf")

  case _ : print("Please enter tour your choice correctly")

print("please select your summarization technique \n press - one for Abstractive \n press - two for Extractive")

tech = input("please enter your choice here")

if(tech == "Abstractive"):

  print("Enter minimum length and max length your want")

  min = int(input("enter  minimum length : "))

  max = int(input("enter maximum length : "))

  summary = Abstractive(text,min,max)

  final_summary = ""

  for i in summary:
     final_summary +=i

  print(final_summary)

elif(tech == "Extractive"):

  summary = Extractive(text)

  print(summary)
else:
  print("Enter input properly")





please select you input format 
 press - one for -string 
 press - two - for - documnet
please enter your choice : one
Enter your text here : Chandrayaan-3 consists of an indigenous Lander module (LM), Propulsion module (PM) and a Rover with an objective of developing and demonstrating new technologies required for Inter planetary missions. The Lander will have the capability to soft land at a specified lunar site and deploy the Rover which will carry out  in-situ chemical analysis of the lunar surface during the course of its mobility. The Lander and the Rover have scientific payloads to carry   out experiments on the lunar surface. The main function of PM is to carry the LM from launch vehicle injection till final lunar 100 km circular    polar orbit and separate the LM from PM. Apart from this, the Propulsion Module also has one scientific payload as a value addition which will    be operated post separation of Lander Module. The launcher identified for Chandrayaan-3 is LVM3 M4 whic

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


the Lander will have the capability to soft land at a specified lunar site. the Propulsion module will carry the LM from launch vehicle injection till final lunar 100 km circular polar orbit. the Lander and the Rover have scientific payloads to carry out experiments on the lunar surface. the launcher identified for Chandrayaan-3 is lvm3m4. it will place the integrated module in an Elliptic Parking Orbit of size 170 x 36500 km.
